In [9]:
from ultralytics import YOLO
from PIL import Image
import cv2


In [ ]:
model = YOLO("Yolo/yolov8s.pt")
ret = model('E:/do-an/images/multibottle/chuan/z4229996124844_43500e5ce281f560b4ac860cc2716092.jpg', save=True,  save_txt=True, save_conf=True)
print("FINSIH")


image 1/1 E:\do-an\images\multibottle\chuan\z4229996124844_43500e5ce281f560b4ac860cc2716092.jpg: 640x480 3 bottles, 3 vases, 172.0ms
Speed: 1.0ms preprocess, 172.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict5
1 label saved to runs\detect\predict5\labels


In [13]:
ret['boxes']

TypeError: list indices must be integers or slices, not str

In [ ]:
from glob import glob
paths = glob(r"E:\do-an\images\multibottle\**\*.jpg")
print(len(paths))
for path in paths:
    model(path, save=True,  save_txt=True, save_conf=True)


In [98]:
import matplotlib.pyplot as plt
import os

def get_coord(img_path :str, txt_path :str):
    # Lay toa do doi tuong 39
    img = cv2.imread(img_path)
    dh, dw, _ = img.shape
    fl = open(txt_path, 'r')
    data = fl.readlines()   
    fl.close()

    for dt in data:

        # Split string to float
        _, x, y, w, h, acc = map(float, dt.split(' '))
        if acc < 0.7:
            return

        # Taken from https://github.com/pjreddie/darknet/blob/810d7f797bdb2f021dbe65d2524c2ff6b8ab5c8b/src/image.c#L283-L291
        # via https://stackoverflow.com/questions/44544471/how-to-get-the-coordinates-of-the-bounding-box-in-yolo-object-detection#comment102178409_44592380
        l = int((x - w / 2) * dw)
        r = int((x + w / 2) * dw)
        t = int((y - h / 2) * dh)
        b = int((y + h / 2) * dh)
        
        if l < 0:
            l = 0
        if r > dw - 1:
            r = dw - 1
        if t < 0:
            t = 0
        if b > dh - 1:
            b = dh - 1
        b_nap = (int)((b-t)/8) + t
        t_nhan = (int)((b-t)/8) + t
        print(os.path.basename(img_path))

        cv2.imwrite('E:\\do-an\\images\\multibottle\\training\\nap\\' + os.path.basename(img_path), img[t:b_nap, l:r])
        cv2.imwrite('E:\\do-an\\images\\multibottle\\training\\nhan\\' + os.path.basename(img_path), img[t_nhan:b, l:r])
        return img[t:b, l:r]
        # img.crop[]

# img = get_coord('./images/multibottle/chuan/z4229996124844_43500e5ce281f560b4ac860cc2716092.jpg', r'E:\do-an\runs\detect\predict\labels\z4229996124844_43500e5ce281f560b4ac860cc2716092.txt')
# cv2.imwrite( "test.png", img)

In [2]:
import os

import numpy as np  
from PIL import Image 
from glob import glob

classes = ['nap', 'nhan', 'thieu_nap', 'thieu_nhan']
# Crop image to train model classify
paths = glob(r"./runs/detect/predict/labels/*.txt")
img = []
for path in paths:
    base_name = os.path.basename(path).split('.')[0]
    coord_nap = []
    coord_nhan = []
    for class_name in classes:
        image_path = "./images/multibottle/"+class_name + "/"+base_name+".jpg"
        if os.path.exists(image_path):
            print("Img path : " , image_path)
            get_coord(image_path, path)
    # get coord vi txt file
    
            
    # img_nap = img.
    # break

# img.resize([200,200])

In [ ]:
img_crop 

In [8]:
import tensorflow as tf
import numpy as np
from src.training.load_model import load_model
from src.training.load_image import get_dataset

checkpoint_dir = "runs/training/"
# Load model
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest
# Create a new model instance
model = load_model(4)

# Load the previously saved weights
model.load_weights(latest)


In [2]:

from glob import glob
testing_image = glob("./images/multibottle/testing/*.jpg")
for path in testing_image:
    # Evaluate
    img = tf.keras.utils.load_img(
        path, target_size=(244, 244)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    print(np.max(img_array[0]), np.min(img_array[0]))
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    # _,_,class_names = get_dataset()
    
    print(path)

    class_names = ['nap', 'nhan', 'thieu_nap', 'thieu_nhan']
    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )


255.0 0.0
1/1 [==============================] - 5s 5s/step
./images/multibottle/testing\z4229996229742_5169139970ea706caa560df927b400fc.jpg
This image most likely belongs to nap with a 47.54 percent confidence.
255.0 0.0
1/1 [==============================] - 0s 25ms/step
./images/multibottle/testing\z4229996237355_e86249b1e89dbadca6d65c1196011776.jpg
This image most likely belongs to nap with a 47.54 percent confidence.
255.0 0.0
1/1 [==============================] - 0s 22ms/step
./images/multibottle/testing\z4229996249916_dccc04e82340cd926ea84605ce7b5a8c.jpg
This image most likely belongs to nap with a 47.54 percent confidence.
255.0 26.0
1/1 [==============================] - 0s 21ms/step
./images/multibottle/testing\z4230020025736_ad5c296bcaf343c85c6a0777e0b59706.jpg
This image most likely belongs to thieu_nap with a 47.33 percent confidence.
255.0 0.0
1/1 [==============================] - 0s 20ms/step
./images/multibottle/testing\z4230030853056_071f2760ec1755674a58bb6425fa4d7a.

In [2]:
import tensorflow as tf
import numpy as np
path = './images/multibottle/testing/z4230031125486_b60dd61bd22b92fd61cdb876471235cc.jpg'
img = tf.keras.utils.load_img(
        path, target_size=(244, 244)
    )
np.asarray(img).flatten()

array([118, 137, 144, ..., 111, 206, 252], dtype=uint8)

In [2]:
import tensorflow as tf
import numpy as np
from src.training.load_model import load_model
from src.common import get_crop_image
import sys
from PyQt6 import QtCore, QtGui, QtWidgets
from PyQt6 import uic
from PyQt6.QtWidgets import (QMainWindow, QTextEdit,
        QFileDialog, QApplication)
from PyQt6.QtGui import QPixmap


from PyQt6.QtCore import *
from PyQt6.QtGui import *
from PyQt6.QtWidgets import QWidget

import cv2 as cv
from ultralytics import YOLO
from collections import deque


# Load Yolo model
yolo_model = YOLO("Yolo/yolov8s.pt")

checkpoint_dir = "runs/training/"
# Load model 
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest
# Create a new model instance
model = load_model(4)

# Load the previously saved weights
model.load_weights(latest)


SyntaxError: invalid syntax (14300173.py, line 60)

In [50]:


        

ret , img = predict("E:/do-an/images/multibottle/img_org/thieu_nap/z4230019226618_434ae9989122909c759c2d19139f939f.jpg")
cv.imwrite("test.png", img)


image 1/1 E:\do-an\images\multibottle\img_org\thieu_nap\z4230019226618_434ae9989122909c759c2d19139f939f.jpg: 640x480 3 bottles, 3 vases, 196.7ms
Speed: 2.0ms preprocess, 196.7ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict42
5 labels saved to runs\detect\predict42\labels


(3, 244, 244, 3)
1/1 [==============================] - 0s 115ms/step


True

In [14]:
predictions_nap

NameError: name 'predictions_nap' is not defined